## 트리의 앙상블

In [3]:
# 정형데이터에 가장 뛰어난 성능을 보이는 모델들 입니다.
# 앙상블 모델들은 결정트리(Decision Tree)를 기반으로 만들어졌습니다.
# 앙삽르 모델들..
# - 랜덤포레스트(Random Forest)
# - 엑스트라 트리(Extra Trees)
# - 그레디언트 부스팅(Gradient Boosting)
# - 히스토그램 기반 그레디언트 부스팅(Histogram-base Gradient Boosting)

## 랜덤포레스트(Random Forest)

In [4]:
### 앙상블 모델 중에 가장 대표격 모델로 사용됨
# - 안정적인 성능으로 널리 사용됨
# - 앙상블 모델 중에 가장 먼저 시도하는 모델이라고 보면 됩니다.
# - 훈련데이터에서 과대적합되는 것을 막아줍니다.
# - 검증데이터와 테스트데이터에서 안정적인 성능을 얻을 수 있음

### 학습 개념
# - 결정트리 하나하나를 랜덤하게 만들어서 숲을 만든다고 보시면 됩니다.
# - 훈련데이터에서 랜덤하게 샘플을 추출하여 훈련을 완료한 후
#   다시 원본 훈련데이터에 반환을 합니다.
# - 랜덤하게 추출 시 이전에 사용된 샘플을 사용할 수도 있음
#   (중복을 허용함)

### 부트스트랩 샘플(Bootstrap Sample0)
# - 위에 설명한 랜덤한 샘플 추출 시 중복을 허용하여 데이터를 샘플링 하는 방식
# - 샘플 추출 바익
# 1. 원본에서 랜덤 샘플 추출
# 2. 훈련 이후 사용이 끝나면 원본에 반환
# 3. 다시 원본에서 샘플 추출, 이때 중복 값 추출 될 수도 있음
# 위 순서를 반복하면서 샘플링을 통해 훈련하는 방식을 랜덤포레스트가 적용하고 있음

### *** 랜덤포레스트는 교차검증을 허용 합니다. ***

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [6]:
### 08_wine.csv 읽어들이고, 훈련 및 테스트 데이터까지 생성해 주세요..
file_path = './data/08_wine.csv'
wine = pd.read_csv(file_path)
wine

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0
...,...,...,...,...
6492,11.2,1.6,3.27,1.0
6493,9.6,8.0,3.15,1.0
6494,9.4,1.2,2.99,1.0
6495,12.8,1.1,3.34,1.0


In [7]:
wine_data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
wine_target = wine['class'].to_numpy()

In [8]:
train_input, test_input, train_target, test_target = \
train_test_split(wine_data, wine_target, random_state=42)

print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

(4872, 3) (4872,)
(1625, 3) (1625,)


## 훈련모델 생성하기

In [9]:
# 랜덤포레스트 패키지 : sklearn.ensemble
# 랜덤포레스트 클래스(모델) : RandomForestClassifier
# 교차검증 패키지 : sklearn.model_selection
# 교차검증 : cross_validate
# 교차검증 후 훈련검증결과와 테스트검증결과 확인하기.

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

In [11]:
rf = RandomForestClassifier(n_jobs = -1, random_state=42)

In [12]:
scores = cross_validate(rf, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
scores

{'fit_time': array([0.24580169, 0.2524488 , 0.22632861, 0.25470209, 0.32727528]),
 'score_time': array([0.04266071, 0.0346837 , 0.04682708, 0.03512597, 0.01695895]),
 'test_score': array([0.88      , 0.90051282, 0.90349076, 0.89014374, 0.88295688]),
 'train_score': array([0.99743392, 0.99692071, 0.99846075, 0.99820421, 0.99820421])}

In [13]:
# 최종 훈련평가 결과 및 검증결과
print(scores['train_score'].mean())
print(scores['test_score'].mean())

0.997844759088341
0.8914208392565683


In [14]:
rf.fit(train_input, train_target)
# 특성중요도 조회하기
# 랜덤 포레스트는 각 특성을 골고루 사용함
rf.feature_importances_

array([0.23155241, 0.49706658, 0.27138101])

In [15]:
### oob 기능 사용
# 훈련에 참여하지 못한 잔여샘플 사용하는 기능
# 기본은 사용안함,
rf = RandomForestClassifier(oob_score = True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8981937602627258


## 엑스트라 트리(Extra Tree)

In [16]:
# - 랜덤포레스트와 유사하게 작동
# - 기본적으로 100개의 결정트리를 훈련함
# - 랜덤포레스트와의 차이점
# : 부트스트랩 샘플링을 지원하지 않음
# : 훈련데이터 전체를 이용하여 결정트리를 생성
# : 무작위로 트리를 분리함
# - 사용되는 속성 : splitter = 'random' 무작위속성
# - 장점
# : 과대적합을 막고, 검증데이터의 평가 값을 높일 수 있음
# : 특성 데이터가 많지 않은 경우에는 랜덤포레스트와 큰 차이가 없음
# - 랜덤포레스트는 불순도 등 여러가지 조건에 따라 결정트리를 생성하기 때문에
#   속도가 느린 반면에,
# - 엑스트라트리는 랜덤하게 결정트리를 생성하기에 속도가 다소 빠르다는 장점있음

In [17]:
### 사용패키지 : 랜덤포레스트와 동일
# 사용되는 클래스(모델) : ExtraTreesClassifier

### 코어 전체사용, train 및 test 결과값 출력
### 교차검증 결과인 train 및 test 결과 확인해 주세요.

In [18]:
from sklearn.ensemble import ExtraTreesClassifier

In [19]:
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)

In [20]:
scores = cross_validate(et, train_input, train_target, \
                        return_train_score = True, n_jobs = -1)
scores

{'fit_time': array([0.15869212, 0.16670156, 0.14089012, 0.18645   , 0.19135976]),
 'score_time': array([0.02730465, 0.02602482, 0.03419304, 0.0299046 , 0.03008032]),
 'test_score': array([0.89128205, 0.89128205, 0.89938398, 0.88706366, 0.88295688]),
 'train_score': array([0.99743392, 0.99692071, 0.99846075, 0.99820421, 0.99820421])}

In [21]:
print(scores['train_score'].mean())
print(scores['test_score'].mean())

0.997844759088341
0.8903937240035804


In [22]:
et.fit(train_input, train_target)
et.feature_importances_

array([0.20702369, 0.51313261, 0.2798437 ])

## 그레디언트 부스팅(Gradient Boosting)

In [23]:
# 깊이(max_depth)가 얕은 결정트리를 사용함
# - 기본적으로 max_depth = 3을 사용
# = 결정트리는 100개 사용
### **** 기존에 다른 훈련모델의 결과가 좋지 않을 때 사용하는 모델 ****
# 기존 훈련모델의 오차를 많이 보완해 줍니다.
# 성능 향상을 위한 모델로 주로 사용됩니다.
# 과대적합에 강하며, 일반화(과대/과소적합이 없는 상태)에 강합니다.

# 성능향상 테스트 방법
# - 결정트리의 갯수를 조절하면서 테스트 진행
# - 학습률을 지원하기 때문에 학습률의 값을 증가시키면서 테스트 진행
#   : 기본 학습률은 0.1

# 단점
# - 순서대로 트리를 추가(랜덤하지 않음)하지 않기 때문에
# - 훈련 속도는 느림
# - 이런 느린 속도를 개선한 모델이
#   '히스토그램 기반 그레디언트 부스팅' 모델임

## 그레디언트 부스팅 모델 생성

In [24]:
### 사용하는 클래스(모델) :GradientBoostingClassifier
# 객체 생성시 아무것도 안주고 seed값만 줍니다.
# 교차검증시에는 train, test 결과값 출력 합니다.

In [25]:
from sklearn.ensemble import GradientBoostingClassifier

In [26]:
gb = GradientBoostingClassifier(random_state = 42)

In [27]:
score = cross_validate(gb, train_input, train_target, \
               return_train_score = True, n_jobs=-1)
score

{'fit_time': array([0.24703121, 0.26337528, 0.27246761, 0.29498243, 0.282727  ]),
 'score_time': array([0.00702858, 0.00204277, 0.00303578, 0.007056  , 0.        ]),
 'test_score': array([0.86461538, 0.87794872, 0.88090349, 0.8613963 , 0.87268994]),
 'train_score': array([0.89299461, 0.88555299, 0.88660852, 0.89276552, 0.88943048])}

In [28]:
print(score['train_score'].mean())
print(score['test_score'].mean())

0.8894704231708938
0.8715107671247301


In [29]:
gb.fit(train_input, train_target)
gb.feature_importances_

array([0.12517641, 0.73300095, 0.14182264])

## 학습률 적용하기

In [30]:
# : 학습률이 커지면 트리 보정을 강하게 하기 때문에,
#   복잡한 모델을 만들어서 일반화 성능을 떨어뜨리게 된다.
# 학습률 : learning_rate=0.1 기본값...

gb = GradientBoostingClassifier(n_estimators = 100,
                                learning_rate = 0.1,
                                random_state = 42)

score = cross_validate(gb, train_input, train_target, \
               return_train_score = True, n_jobs=-1)
score

{'fit_time': array([0.24377608, 0.28320765, 0.27103209, 0.25127316, 0.25868988]),
 'score_time': array([0.        , 0.00702095, 0.00099874, 0.        , 0.        ]),
 'test_score': array([0.86461538, 0.87794872, 0.88090349, 0.8613963 , 0.87268994]),
 'train_score': array([0.89299461, 0.88555299, 0.88660852, 0.89276552, 0.88943048])}

In [31]:
print(score['train_score'].mean())
print(score['test_score'].mean())

0.8894704231708938
0.8715107671247301


## 히스토그램 기반 그레디언트 부스팅
##### - Histogram_base Gradient Boosting

In [32]:
## 사용하는 클래스(모델) : HistGradientBoostingClassifier

In [64]:
from sklearn.ensemble import HistGradientBoostingClassifier

In [65]:
hgb = HistGradientBoostingClassifier(random_state=42)

In [66]:
scores = cross_validate(hgb, train_input, train_target, \
                        return_train_score = True, n_jobs = -1)
scores

{'fit_time': array([0.12248969, 0.12335443, 0.14288497, 0.1366117 , 0.11931729]),
 'score_time': array([0.0009985 , 0.00465178, 0.00802088, 0.00823569, 0.00201225]),
 'test_score': array([1.        , 0.92592593, 0.92592593, 0.92307692, 1.        ]),
 'train_score': array([1., 1., 1., 1., 1.])}

In [67]:
print(scores['train_score'].mean())
print(scores['test_score'].mean())

1.0
0.9549857549857551


In [68]:
hgb.fit(train_input, train_target)
hgb.score(test_input, test_target)

0.9777777777777777

## 사이킷런 이외 다른 패키지에서 지원하는
## 히스토그램 기반 그레디언트 부스팅 기능 모델들

## XGBoost

In [38]:
from xgboost import XGBClassifier

In [39]:
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, \
                        return_train_score = True, n_jobs = -1)
scores

{'fit_time': array([0.11939478, 0.12039256, 0.09842801, 0.08450651, 0.08800364]),
 'score_time': array([0.00099778, 0.00806308, 0.        , 0.0175488 , 0.01568651]),
 'test_score': array([0.87487179, 0.89333333, 0.8973306 , 0.87268994, 0.8788501 ]),
 'train_score': array([0.96176546, 0.96279189, 0.95997948, 0.95690097, 0.9656234 ])}

In [40]:
print(scores['train_score'].mean())
print(scores['test_score'].mean())

0.9614122399872658
0.8834151529510873


## LightGBM

In [41]:
# - 마이크로소프트에서 만든 히스토그램 기반 그레디언트 부스트 패키지
# - 훈련 속도가 매우 빠름
# - 최신 기술을 많이 적용하고 있어서, 인기가 올라가고 있음
# - 

In [42]:
# conda install -c conda-forge lightgbm
# pip install lightgbm

In [43]:
from lightgbm import LGBMClassifier

In [44]:
lg = LGBMClassifier(random_state=42)

In [46]:
scores = cross_validate(lg, train_input, train_target, \
                       return_train_score = True, n_jobs=-1)
scores

{'fit_time': array([0.03670454, 0.05293918, 0.05568457, 0.06076908, 0.04601431]),
 'score_time': array([0.01566529, 0.00568104, 0.        , 0.00613189, 0.01716137]),
 'test_score': array([0.87487179, 0.89846154, 0.90862423, 0.85934292, 0.88193018]),
 'train_score': array([0.93892738, 0.94277649, 0.94125192, 0.93996921, 0.94381734])}

In [47]:
print(scores['train_score'].mean())
print(scores['test_score'].mean())

0.9413484712095832
0.8846461327857632


## 와인데이터로 오늘 배운 내용들 실습

In [60]:
import pandas as pd
import numpy as np

In [45]:
from sklearn.datasets import load_wine

wine_all = load_wine()
wine_all

{'data': array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
         1.065e+03],
        [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
         1.050e+03],
        [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
         1.185e+03],
        ...,
        [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
         8.350e+02],
        [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
         8.400e+02],
        [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
         5.600e+02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [52]:
wine_input = wine_all['data']
wine_target = wine_all['target']

In [61]:
from sklearn.model_selection import train_test_split

In [59]:
train_input, test_input, train_target, test_target = \
train_test_split(wine_input, wine_target, random_state = 42)

In [73]:
print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

(133, 13) (133,)
(45, 13) (45,)


## 결정트리 모델을 이용해서 교차검증

In [69]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate

In [75]:
dt = DecisionTreeClassifier(random_state = 42)

In [76]:
scores = cross_validate(dt, train_input, train_target, \
                       return_train_score = True, n_jobs=-1)
scores

{'fit_time': array([0.00099373, 0.00199652, 0.00099874, 0.00199342, 0.00099707]),
 'score_time': array([0.00099778, 0.        , 0.00099635, 0.        , 0.        ]),
 'test_score': array([0.96296296, 0.92592593, 0.92592593, 0.96153846, 0.92307692]),
 'train_score': array([1., 1., 1., 1., 1.])}

In [77]:
print(scores['train_score'].mean())
print(scores['test_score'].mean())

1.0
0.93988603988604
